In [1]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
conn.execute('''
        CREATE TABLE Foreign_book(
            id INTEGER,
            title TEXT,
            genres TEXT,
            author TEXT,
            rating REAL, 
            publisher TEXT, 
            p_date INTEGER, 
            pages INTEGER, 
            description TEXT, 
            imgUrl TEXT
        )
''')
conn.commit()
conn.close()

In [2]:
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
conn.execute("DROP TABLE foreign_book")
conn.commit()
conn.close()